In [1]:
import torch.nn.functional as f
from torch.nn import BatchNorm2d,Conv2d
from torch import nn

In [3]:
class DnCNN(nn.Module):
  def __init__(self,in_channels,out_channels,depth):
    super(DnCNN, self).__init__()
    #First layer
    #convolution, then ReLU
    self.c1=Conv2d(in_channels=in_channels,out_channels=64,stride=1,padding=1,kernel_size=3,bias=False)
    #For next layers
    #Also relu
    self.c2=Conv2d(in_channels=64,out_channels=64,stride=1,padding=1,kernel_size=3)
    self.b2=BatchNorm2d(64)
    #For last layer
    self.c3=Conv2d(in_channels=64,out_channels=out_channels,stride=1,padding=1,kernel_size=3)
    self.dep = depth

  def forward(self,x):
    x=self.c1(x)
    x=f.relu(x, inplace=True)
    for i in range(self.dep-2):
      x=self.c2(x)
      x=self.b2(x)
      x=f.relu(x, inplace=True)
    x=self.c3(x)
    return x

In [3]:
#Set at the start just so there is no issue
"""
y is the noisy image and it is used as input

x is the noiseless image and it is used as a target label
"""
class loss(nn.Module): #N is dataset size
  def __init__(self):
    super(loss,self).__init__()
  def forward(self,out,y,x):
    return torch.norm(out-(y-x))/(2*N) #Where does N get defined?

In [4]:
model=DnCNN(1,1,10)
model.cuda()

DnCNN(
  (c1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (c2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (b2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (c3): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [ ]:
from torch import optim
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)

In [ ]:
#Tensor to data loader
def training_function(net, train_loader, optim, loss, epochs=50):
  loss_list=[]

  net.train()
  train_loss = []
  for e in range(epochs):
    for i, train_img in enumerate(train_loader):
      noisy, clean = train_img
      print(len(noisy))
      noisy = noisy.cuda()
      clean = clean.cuda()
      optim.zero_grad()

      # forward + backward
      pred_mask = net(noisy)
      loss_val = loss(pred_mask, noisy, clean)
      optim.step()
      loss_val.backward()

      train_loss += [loss_val.item()] * input.shape[0]

    print ("Epoch {}: [{}/{}] Loss: {:.3f}".format(e, len(train_loader), len(train_loader),np.mean(train_loss)))
  
  return np.mean(train_loss)

In [ ]:
#Now that model is defined
#We can write a new code to take dataloader and run training

